# Introduzione all'Intelligenza Artificiale<br/>Supporto per le Esercitazioni


Questo notebook rappresenta una guida all'implementazione e utilizzo dei principali algoritmi e modelli presentati nell'ambito del corso "Introduzione all'Intelligenza Artificiale", corso di Laurea in Informatica, Università di Pisa, anno accademico 2024/2025.

Il codice presentato di seguito è un'implementazione delle funzioni riportate nel testo di riferimento del corso:<br/>
*S. Russell, P. Norvig, "Artificial Intelligence: A Modern Approach, Pearson, 4th Edition, 2020*

Ulteriori risorse possono essere reperite online all'indirizzo: https://github.com/aimacode

# Algoritmi di Ricerca

Di seguito, verrano presentati i principali algoritmi per risolvere problemi di ricerca.

## Classi e Strutture Dati
Introduciamo le astrazioni necessarie per poter rappresentare e risolvere un problema di ricerca.<br/>
Queste classi saranno i *building blocks* per poter successivamente implementare gli algoritmi risolutori.

In [ ]:
class Problem:
    # Costrutture. Specifica lo stato iniziale e lo stato (o la lista di stati) obiettivo
    def __init__(self, initial_state=None, goal_state=None):
        self.initial_state = initial_state
        self.goal_state = goal_state

    # Metodo astratto, deve essere implementato in ogni sottoclasse che specializza Problem
    # Dato lo stato 'state', restituisce la lista di azioni che possono essere eseguite
    def actions(self, state): raise NotImplementedError

    # Metodo astratto, deve essere implementato in ogni sottoclasse che specializza Problem
    # Dato lo stato 'state' e l'azione 'action', restituisce lo stato risultante in base al modello di transizione del problema
    def result(self, state, action): raise NotImplementedError

    # Dato lo stato 'state', restituisce True se è lo stato obiettivo, False altrimenti
    def is_goal(self, state):
        # controllo se self.goal_state è una lista, quindi esistono molteplici stati obiettivi
        if isinstance(self.goal_state, list):
            return state in self.goal_state
        # altrimenti, self.goal_state è un solo elemento, quindi esiste un solo stato obiettivo
        else: return state == self.goal_state

    # Restituisce il costo dell'esecuzione dell'azione 'action' che porti dallo 'stateA' allo 'stateB'
    # Questa implementazione di default prevede costo unitario per ogni azione
    def action_cost(self, action, stateA, stateB): return 1

    def h(self, node): return 0

### Esempio di utilizzo della classe Problem, per la rappresentazione di un problema specifico

Per rappresentare un problema concreto, occorre specializzare la classe Problem, implementanto almeno le funzioni *actions* e *result*.<br/>
Un esempio semplice di problema, la cui mappa degli stati è raffigurata in figura, è definito nella classe *ToyProblem1*
![](https://github.com/luigiquara/IIA-python/blob/main/img/toy_problem1.png)

### Come rappresentare un nodo nell'albero di ricerca - La classe Node

In [ ]:
class Node:
    # Costruttore. Specifica lo stato associato al nodo, il nodo padre,
    # l'azione che ha generato il nodo, il costo del cammino dalla radice al nodo
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    # Specifica la stringa da stampare per rappresentare il nodo
    def __repr__(self):
        description = f'state: {self.state} - path cost: {self.path_cost}'
        return description

    # Restituisce la profondità del nodo
    # Se il nodo corrente non ha nodo padre, i.e. self.parent is None, la profondità è 0
    # Altrimenti, la profondità è quella del nodo padre più 1
    def __len__(self): return 0 if self.parent is None else return 1 + len(self.parent)

    # Restituisce la lista delle azioni compiute per arrivare al nodo corrente
    # Se il nodo corrente non ha nodo padre, i.e. self.parent is None, nessuna azione è stata ancora eseguita
    def path_actions(self):
        if self.parent is None: actions = []
        else: actions = self.parent.path_actions().append(self.action)
        return actions

    # Restituisce la lista degli stati attraversati per arrivare al nodo corrente
    # Se il node corrente non ha nodo padre, i.e. self.parent is None, la lista degli stati corrisponde allo stato attuale
    def path_states(self):
        if self.parent is None: states = self.state
        else: states = self.parent.path_states().append(self.state)
        return states

    # Funzione che espande un nodo, generando tutti i nodi figli
    # La keyword "yield" permette di creare un generatore, che restituisce un nodo figlio alla volta
    def expand(self, problem):
        s = self.state
        # per ogni possibile azione nello stato corrente
        for action in problem.actions(s):
            s1 = problem.result(s, action)
            cost = self.path_cost + problem.action_cost(action, s, s1)
            yield Node(s1, self, action, cost)